# Download data

In [2]:
from getpass import getpass
import os
user = 'ahmadelsallab'
key = '6b7ffe97ff5bc0656e325b746b72fa31'

if '.kaggle' not in os.listdir('/root'):
    !mkdir ~/.kaggle
!touch /root/.kaggle/kaggle.json
!chmod 666 /root/.kaggle/kaggle.json
with open('/root/.kaggle/kaggle.json', 'w') as f:
    f.write('{"username":"%s","key":"%s"}' % (user, key))
!chmod 600 /root/.kaggle/kaggle.json


!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification
!chmod 777 *
!ls

  0% 0.00/221k [00:00<?, ?B/s]
100% 221k/221k [00:00<00:00, 56.4MB/s]
 74% 9.00M/12.1M [00:00<00:00, 16.3MB/s]
100% 12.1M/12.1M [00:00<00:00, 22.4MB/s]
 99% 271M/273M [00:06<00:00, 38.9MB/s]
100% 273M/273M [00:06<00:00, 42.9MB/s]
sample_data  sample_submission.csv.zip	test.csv.zip  train.csv.zip


In [3]:
!unzip sample_submission.csv.zip
!unzip test.csv.zip
!unzip train.csv.zip
!ls

Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               
sample_data	       sample_submission.csv.zip  test.csv.zip	train.csv.zip
sample_submission.csv  test.csv			  train.csv


In [4]:
# Mount GDrive
from google.colab import drive
drive.mount('/content/gdrive')
gdrive_path = 'gdrive/My Drive/Colab Notebooks/JIGSAW/dat'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


# Explore data samples

In [0]:
# Data
trn_limit = -1
tst_limit = -1

In [6]:
import pandas as pd
train_df = pd.read_csv('train.csv')[:trn_limit]
train_df.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.0,4,47


In [7]:
import pandas as pd
test_df = pd.read_csv('test.csv')[:tst_limit]
test_df.head()

,id,comment_text
0,7000000,Jeff Sessions is another one of Trump's Orwell...
1,7000001,I actually inspected the infrastructure on Gra...
2,7000002,No it won't . That's just wishful thinking on ...
3,7000003,Instead of wringing our hands and nibbling the...
4,7000004,how many of you commenters have garbage piled ...


# Data pre-processing

In [0]:
import numpy as np

In [0]:
trn_texts = list(train_df['comment_text'])
tst_texts = list(test_df['comment_text'])
texts = trn_texts + tst_texts

In [10]:
trn_texts[:10]

["This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!",
 "Thank you!! This would make my life a lot less anxiety-inducing. Keep it up, and don't let anyone get in your way!",
 'This is such an urgent design problem; kudos to you for taking it on. Very impressive!',
 "Is this something I'll be able to install on my site? When will you be releasing it?",
 'haha you guys are a bunch of losers.',
 'ur a sh*tty comment.',
 'hahahahahahahahhha suck it.',
 'FFFFUUUUUUUUUUUUUUU',
 'The ranchers seem motivated by mostly by greed; no one should have the right to allow their animals destroy public land.',
 "It was a great show. Not a combo I'd of expected to be good together but it was."]

In [11]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence

tokenizer = Tokenizer()


Using TensorFlow backend.


In [0]:
tokenizer.fit_on_texts(texts)


In [0]:
del texts
del tst_texts
del test_df

In [0]:
binary_trn_texts = tokenizer.texts_to_sequences(trn_texts)

In [15]:

maxlen = max([len(s) for s in binary_trn_texts])
maxlen
#idx = np.argmax([len(s) for s in binary_trn_texts])
#trn_texts[idx]

317

In [0]:
from keras.preprocessing.sequence import pad_sequences

x_train = np.array(pad_sequences(binary_trn_texts, maxlen=maxlen, padding='post', truncating='post'))


In [0]:
del binary_trn_texts

In [18]:
x_train.shape

(1804873, 317)

In [19]:
x_train[:10]

array([[    18,      6,     34, ...,      0,      0,      0],
       [   406,     10,     18, ...,      0,      0,      0],
       [    18,      6,    149, ...,      0,      0,      0],
       ...,
       [195872,      0,      0, ...,      0,      0,      0],
       [     1,  12600,    393, ...,      0,      0,      0],
       [    12,     22,      5, ...,      0,      0,      0]], dtype=int32)

In [20]:
#y_train = np.array(list(train_df['target']))
y_train = np.array((list((train_df['target']>0.5)*1)))

y_train.shape

(1804873,)

In [21]:
y_train[:10]

array([0, 0, 0, 0, 1, 1, 0, 0, 0, 0])

In [22]:

# Shuffle
'''
np.random.seed(42)
trn_idx = np.random.permutation(len(trn_texts))

x_train = x_train[trn_idx]
y_train = y_train[trn_idx]
'''


'\nnp.random.seed(42)\ntrn_idx = np.random.permutation(len(trn_texts))\n\nx_train = x_train[trn_idx]\ny_train = y_train[trn_idx]\n'

In [23]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42, shuffle=True)
'''
trn_idx = int(0.9*len(trn_texts))

x_val = x_train[trn_idx:]
y_val = y_train[trn_idx:]
x_train = x_train[:trn_idx]
y_train = y_train[:trn_idx]
'''

'\ntrn_idx = int(0.9*len(trn_texts))\n\nx_val = x_train[trn_idx:]\ny_val = y_train[trn_idx:]\nx_train = x_train[:trn_idx]\ny_train = y_train[:trn_idx]\n'

In [24]:
print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(1624385, 317)
(180488, 317)
(1624385,)
(180488,)


# Model

In [0]:
model_name = 'lstm_model_1'

In [26]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM, GRU, Bidirectional
from keras.layers import Conv1D, MaxPooling1D

vocab_size = len(tokenizer.word_index)

# Model
# Embedding
embedding_size = 300

# LSTM
lstm_output_size = 100



# Model 1: Simple LSTM, with Embedding mask_zero
model = Sequential()
model.add(Embedding(vocab_size+1, embedding_size, mask_zero=True, input_length=maxlen))
model.add(LSTM(lstm_output_size))
model.add(Dropout(0.2))
#model.add(Dense(500))
#model.add(Dropout(0.25))
#model.add(Dense(200))
#model.add(Dropout(0.25))
#model.add(Dense(100))
#model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))


'''
# Model 2: BiGRU
sentence_input = Input(shape=(maxlen,), dtype='int32')
embedded_sequences = Embedding(vocab_size+1, embedding_size, mask_zero=True)(sentence_input)
l_lstm = Bidirectional(GRU(300))(embedded_sequences)
l_dense = Dense(100, activation='relu')(l_lstm)
output = Dense(1, activation='sigmoid')(l_dense)
model = Model(sentence_input, output)
'''

'''
# Model 3: BiGRU + distributed dense
sentence_input = Input(shape=(maxlen,), dtype='int32')
embedded_sequences = Embedding(vocab_size+1, embedding_size, mask_zero=True)(sentence_input)
l_lstm = Bidirectional(GRU(1000, return_sequences=True))(embedded_sequences)
l_word_enc = TimeDistributed(Dense(500))(l_lstm)
l_lstm_2 = LSTM(500)(l_word_enc)
l_dense = Dense(200, activation='relu')(l_word_enc)
l_dense = Dense(100, activation='relu')(l_dense)
output = Dense(1, activation='sigmoid')(l_lstm_2)
model = Model(sentence_input, output)
'''



model.summary()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 317, 300)          123013800 
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 123,174,301
Trainable params: 123,174,301
Non-trainable params: 0
_________________________________________________________________


In [0]:


from keras.callbacks import ModelCheckpoint, Callback
filepath = os.path.join(gdrive_path, 'jigsaw_' + model_name + '.h5')
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')




In [0]:
from keras import optimizers
# Optimizer
lr = 0.001
opt = optimizers.Adam(lr=lr)
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])



In [0]:
from keras.callbacks import Callback
from sklearn.metrics import roc_auc_score, auc
class roc_callback(Callback):
    def __init__(self, x_val, y_val):
        
        self.x_val = x_val
        # y_val is continous, but according to the competetion, the thresh is at 0.5, so we can convert to binary
        # The *1 is to convert False/True into 0/1    
        self.y_val = np.array((y_val>0.5)*1)
        
    
    def on_train_begin(self, logs={}):
        return
 
    def on_train_end(self, logs={}):
        return
 
    def on_epoch_begin(self, epoch, logs={}):
        return
 
    def on_epoch_end(self, epoch, logs={}):        
        y_pred = self.model.predict(self.x_val)

        val_auc  = roc_auc_score(self.y_val, model.predict(x_val))

        
        print('Epoch: ' + str(epoch) + ' AUC: ' + str(val_auc))
        
        return
 
    def on_batch_begin(self, batch, logs={}):
        return
 
    def on_batch_end(self, batch, logs={}):
        return 

In [30]:
print('Train...')
callbacks_lst = [checkpoint, roc_callback(x_val, y_val)]
# Training
batch_size = 1024
epochs = 20
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,          
          validation_data=(x_val, y_val),
          #validation_split=0.2,
          callbacks=callbacks_lst)

Train...
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:107: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 123013800 elements. This may consume a large amount of memory.
  num_elements)


Train on 1624385 samples, validate on 180488 samples
Epoch 1/20
1624385/1624385 [==============================] - 2044s 1ms/step - loss: 0.1201 - acc: 0.9592 - val_loss: 0.0965 - val_acc: 0.9638

Epoch 00001: val_acc improved from -inf to 0.96378, saving model to gdrive/My Drive/Colab Notebooks/JIGSAW/dat/jigsaw_lstm_model_1.h5
Epoch: 0 AUC: 0.963491070202169
Epoch 2/20
1624385/1624385 [==============================] - 2052s 1ms/step - loss: 0.0866 - acc: 0.9665 - val_loss: 0.0952 - val_acc: 0.9640

Epoch 00002: val_acc improved from 0.96378 to 0.96397, saving model to gdrive/My Drive/Colab Notebooks/JIGSAW/dat/jigsaw_lstm_model_1.h5
Epoch: 1 AUC: 0.9661882447932725
Epoch 3/20
1624385/1624385 [==============================] - 2071s 1ms/step - loss: 0.0719 - acc: 0.9719 - val_loss: 0.1024 - val_acc: 0.9625

Epoch 00003: val_acc did not improve from 0.96397


KeyboardInterrupt: ignored

In [31]:
from sklearn.metrics import roc_auc_score, auc
# y_val is continous, but according to the competetion, the thresh is at 0.5, so we can convert to binary
# The *1 is to convert False/True into 0/1
#fpr, tpr, thresh  = roc_auc_score(np.array((y_val>0.5)*1), model.predict(x_val))
#val_auc = auc(fpr, tpr)
val_auc = roc_auc_score(y_val, model.predict(x_val))
val_auc

0.9618356010996979

# Prepare test data

In [0]:
import pandas as pd
test_df = pd.read_csv('test.csv')
test_df.head()
tst_texts = list(test_df['comment_text'])
binary_tst_texts = tokenizer.texts_to_sequences(tst_texts)
x_test = np.array(pad_sequences(binary_tst_texts, maxlen=maxlen))

In [0]:
y_test = model.predict(x_test)

sub_df = pd.DataFrame({'id': test_df['id'], 'prediction':list(y_test)})

sub_df.to_csv('submission_' + model_name + '.csv')
                      


In [0]:
'''
submission = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv', index_col='id')
submission['prediction'] = test_preds
submission.reset_index(drop=False, inplace=True)
submission.head()
'''